In [19]:
import logging
import sys
import nest_asyncio
from llama_index import (
    VectorStoreIndex,
    ServiceContext,
    Response,
    
)
import pinecone
import openai
import streamlit as st
from llama_index.vector_stores import PineconeVectorStore
from llama_index.llms import OpenAI
from llama_index.evaluation import CorrectnessEvaluator

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

nest_asyncio.apply()

In [20]:
openai.api_key = st.secrets.openai_key
api_key = "2bcecfc8-35f9-44dc-9459-ab0cc2781b29"
pinecone.init(api_key=api_key, environment="gcp-starter")
pinecone_index = pinecone.Index("tailwind-hugging")

In [21]:
vector_store = PineconeVectorStore(pinecone_index=pinecone_index)
service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0.5, system_prompt="You are an expert on TailwindCSS and your job is to answer technical questions. Assume that all questions are related to TailwindCSS. Keep your answers technical and based on facts – do not hallucinate features."))
index = VectorStoreIndex.from_vector_store(vector_store=vector_store, service_context=service_context)

In [22]:
evaluator = CorrectnessEvaluator(service_context=service_context)

In [23]:
chat_engine = index.as_chat_engine(chat_mode="reAct", verbose=True)
query = query_str = "Can I use items-red with columns?"
response = chat_engine.chat(query)
reference = "It is a property that allows you to round the corners of an element. You can use it in TailwindCSS by adding the rounded class to an element. For example, rounded-lg will round the corners of an element by 0.5rem."

INFO:llama_index.chat_engine.condense_question:Querying with: Can I use the "items-red" style with columns?
Querying with: Can I use the "items-red" style with columns?
Querying with: Can I use the "items-red" style with columns?
Querying with: Can I use the "items-red" style with columns?
Querying with: Can I use the "items-red" style with columns?
Querying with: Can I use the "items-red" style with columns?


In [24]:
result = evaluator.evaluate(
    query=query,
    response=response.response,
    reference=reference,
)

print("response: ", response.response)
print("result.score: ", result.score)
print("result.feedback: ", result.feedback)

response:  Yes, you can use the "items-red" style with columns.
result.score:  2.0
result.feedback:  The generated answer is relevant to the user query, but it contains a mistake. The correct style name is "rounded" not "items-red".
